In [173]:
import requests
import lxml.html as lh
import pandas as pd

# First get the webpage and find the table (marked in html with /tr)

In [174]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website

page = requests.get(url)

#Store the contents of the website under doc

doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML

tr_elements = doc.xpath('//tr')

# Loop and import the table rows into a python list, find the column names

In [175]:
#Create empty list

col=[]
i=0

#For each row, store each first element (header) and an empty list

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


# Iterate through the list and clean up the data

In [176]:
#Since the first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0 and t!="Not assigned":
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

# Create a python dictionary from the clean data and make a pandas dataframe from that dictionary - shape the data as in the exercise instructions.

In [177]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

df.columns=['Postcode','Borough','Neighbourhood']

df.drop([0],axis=0,inplace=True)

df.reset_index()

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, 
# you will notice that M5A is listed twice and has two neighborhoods: 
# Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods 
# separated with a comma as shown in row 11 in the above table.

df=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, 
# the value of the Borough and the Neighborhood columns will be Queen's Park.

df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

df.shape


(103, 2)

# Then just for the heck of it print the first ten lines of the resulting dataframe.

In [178]:
df.head(10)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern\n,Rouge\n"
M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n"
M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n"
M1G,Scarborough,Woburn\n
M1H,Scarborough,Cedarbrae\n
M1J,Scarborough,Scarborough Village\n
M1K,Scarborough,"Ionview\n,East Birchmount Park\n,Kennedy Park\n"
M1L,Scarborough,"Golden Mile\n,Oakridge\n,Clairlea\n"
M1M,Scarborough,"Cliffcrest\n,Cliffside\n,Scarborough Village W..."
